In [6]:
import os, warnings, numpy as np, pandas as pd, matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

from pathlib import Path

DATA_PATH = Path("../data/raw/data.csv")
SAMPLE_FOR_DEV = 200_000
RANDOM_STATE = 42

ARTIFACTS_DIR = Path("../artifacts")
ARTIFACTS_DIR.mkdir(exist_ok=True)

print(f'Using DATA_PATH="{DATA_PATH.resolve()}"')
print("Artifacts folder:", ARTIFACTS_DIR.resolve())


Using DATA_PATH="/Users/pamudithasenanayake/Downloads/Sliit 3-1/FDM - Fundamentals of Data Mining /FDM_project/data/raw/data.csv"
Artifacts folder: /Users/pamudithasenanayake/Downloads/Sliit 3-1/FDM - Fundamentals of Data Mining /FDM_project/artifacts


In [7]:
assert DATA_PATH.exists(), f"CSV not found at {DATA_PATH}"

df = pd.read_csv(DATA_PATH, low_memory=False)
print("Original shape:", df.shape)
df.head(3)

# Optional: sample for faster prototyping
if SAMPLE_FOR_DEV and len(df) > SAMPLE_FOR_DEV:
    df = df.sample(SAMPLE_FOR_DEV, random_state=RANDOM_STATE).reset_index(drop=True)
    print(f"Downsampled to {len(df):,} rows for dev purposes.")

df.describe(include='all').T.head(20)


Original shape: (302010, 30)
Downsampled to 200,000 rows for dev purposes.


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Transaction_ID,199786.0,NaN,NaN,NaN,5494520.118847,2595422.271277,1000007.0,3246077.75,5500641.5,7734932.5,9999995.0
Customer_ID,199794.0,NaN,NaN,NaN,54994.553525,26001.648657,10000.0,32467.0,54960.5,77497.75,99998.0
Name,199757,119661,Michael Smith,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Email,199766,46605,Michael70@gmail.com,64,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Phone,199757.0,NaN,NaN,NaN,5500601362.951986,2596293808.967086,1000049414.0,3253565390.0,5500959657.0,7747866972.0,9999996122.0
Address,199786,198784,923 Eric Mission Apt. 706,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,199840,130,Chicago,14356,NaN,NaN,NaN,NaN,NaN,NaN,NaN
State,199814,54,England,41703,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zipcode,199793.0,NaN,NaN,NaN,50280.770998,28985.940994,501.0,25360.0,50524.0,75264.0,99949.0
Country,199821,5,USA,63186,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
nulls = df.isna().sum().sort_values(ascending=False)
nulls_df = pd.DataFrame({
    "column": nulls.index,
    "null_count": nulls.values,
    "null_pct": (nulls.values / len(df)) * 100
})
display(nulls_df)


dup_count = df.duplicated().sum()
print(f"Full-row duplicates: {dup_count}")


if "Transaction_ID" in df.columns:
    dup_txn = df.duplicated(subset=["Transaction_ID"]).sum()
    print(f"Duplicate Transaction_ID: {dup_txn}")


dtypes_df = pd.DataFrame(df.dtypes, columns=["dtype"]).reset_index().rename(columns={"index":"column"})
display(dtypes_df.head(30))


,column,null_count,null_pct
0,Year,254,0.1270
1,Amount,245,0.1225
2,Date,244,0.1220
3,Name,243,0.1215
4,Phone,243,0.1215
5,Total_Purchases,235,0.1175
6,Email,234,0.1170
7,Time,232,0.1160
8,Total_Amount,226,0.1130
9,Shipping_Method,222,0.1110


Full-row duplicates: 2
Duplicate Transaction_ID: 3389


,column,dtype
0,Transaction_ID,float64
1,Customer_ID,float64
2,Name,object
3,Email,object
4,Phone,float64
5,Address,object
6,City,object
7,State,object
8,Zipcode,float64
9,Country,object


In [10]:
def to_snake(name):
    return name.strip().replace(" ","_").replace("/","_").replace("-","_").lower()
df.columns = [to_snake(c) for c in df.columns]


if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors='coerce')


for col in ["amount","total_amount","ratings","total_purchases","age"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')


for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].astype(str).str.strip()


from sklearn.impute import SimpleImputer
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in df.columns if c not in num_cols]

df[num_cols] = SimpleImputer(strategy="median").fit_transform(df[num_cols])
df[cat_cols] = SimpleImputer(strategy="most_frequent").fit_transform(df[cat_cols])


before = len(df)
df = df.drop_duplicates().reset_index(drop=True)
print(f"Removed {before - len(df):,} duplicates. New shape: {df.shape}")


Removed 2 duplicates. New shape: (199998, 30)


In [11]:
if "date" in df.columns:
    df["year_num"] = df["date"].dt.year
    df["month_num"] = df["date"].dt.month
    df["dayofweek"] = df["date"].dt.dayofweek
    df["is_weekend"] = df["dayofweek"].isin([5,6]).astype(int)

# Clip extreme amounts
for col in ["amount","total_amount"]:
    if col in df.columns:
        q1, q99 = df[col].quantile([0.01,0.99])
        df[col+"_clipped"] = df[col].clip(q1,q99)

# Simple RFM features
if {"customer_id","date","amount"}.issubset(df.columns):
    last_date = df["date"].max()
    cust_grp = df.groupby("customer_id").agg(
        last_purchase=("date","max"),
        frequency=("transaction_id","nunique") if "transaction_id" in df.columns else ("customer_id","count"),
        monetary=("amount","sum")
    )
    cust_grp["recency_days"] = (last_date - cust_grp["last_purchase"]).dt.days
    df = df.merge(cust_grp[["recency_days","frequency","monetary"]].reset_index(), on="customer_id", how="left")

# Top-K brand/category
if "product_brand" in df.columns:
    top_brands = df["product_brand"].value_counts().index[:30]
    df["product_brand_top"] = np.where(df["product_brand"].isin(top_brands), df["product_brand"], "OTHER")
if "product_category" in df.columns:
    top_cats = df["product_category"].value_counts().index[:30]
    df["product_category_top"] = np.where(df["product_category"].isin(top_cats), df["product_category"], "OTHER")

print("Feature engineering done. Current shape:", df.shape)
df.head(3)


Feature engineering done. Current shape: (199998, 41)


,transaction_id,customer_id,name,email,phone,address,city,state,zipcode,country,...,month_num,dayofweek,is_weekend,amount_clipped,total_amount_clipped,recency_days,frequency,monetary,product_brand_top,product_category_top
0,8973042.0,20082.0,Ronald Williamson,David75@gmail.com,3.740098e+09,4618 Pamela Wells Suite 878,Portsmouth,England,8805.0,UK,...,11,2,0,143.404730,1003.833112,12,4,1275.469378,Zara,Clothing
1,6700726.0,59044.0,Julie Yoder,Adrian66@gmail.com,4.959731e+09,5158 Russell Creek Apt. 082,Darwin,New South Wales,71099.0,Australia,...,1,0,0,103.655237,103.655237,26,4,1145.523341,Apple,Electronics
2,9007686.0,32393.0,Isaac Livingston,Robyn24@gmail.com,6.717169e+09,06198 Stephen Row,Frankfurt,Berlin,83860.0,Germany,...,1,2,0,429.278597,429.278597,50,5,1436.608975,Samsung,Electronics
